In [230]:
%%configure -f
{"name": "gerard-transit-query", "executorMemory": "12G", "numExecutors": 16, "executorCores": 4,
 "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"python3"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
75,application_1586890731024_0071,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
75,application_1586890731024_0071,pyspark,idle,Link,Link,✔


In [2]:
import matplotlib.pyplot as plt
import subprocess

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
70,application_1586890731024_0066,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def list_hdfs(dir):
    cmd = '/opt/hadoop/bin/hdfs dfs -ls {}'.format(dir)
    return subprocess.check_output(
        cmd,
        stderr = subprocess.STDOUT,
        shell=True).decode('utf-8')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [229]:
import time
hdfs_dir = 'hdfs:///user/transit/*/*/*.json'
print(list_hdfs(hdfs_dir))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-rw-r--r--   2 root supergroup          0 2020-04-23 20:39 hdfs:///user/transit/2020-04-23/1587674347550/part-00000-8ff794e5-a1c2-4f3f-a9dc-041c007e45ce-c000.json
-rw-r--r--   2 root supergroup      75859 2020-04-23 20:39 hdfs:///user/transit/2020-04-23/1587674347550/part-00109-8ff794e5-a1c2-4f3f-a9dc-041c007e45ce-c000.json
-rw-r--r--   2 root supergroup          0 2020-04-23 21:02 hdfs:///user/transit/2020-04-23/1587675718410/part-00000-785eb475-761f-4f73-97f2-8c029a30e3fa-c000.json
-rw-r--r--   2 root supergroup      77776 2020-04-23 21:02 hdfs:///user/transit/2020-04-23/1587675718410/part-00015-785eb475-761f-4f73-97f2-8c029a30e3fa-c000.json
-rw-r--r--   2 root supergroup          0 2020-04-23 21:02 hdfs:///user/transit/2020-04-23/1587675731941/part-00000-c46d7a7b-8d95-41f4-8528-2a2beb1a5021-c000.json
-rw-r--r--   2 root supergroup      77561 2020-04-23 21:02 hdfs:///user/transit/2020-04-23/1587675731941/part-00015-c46d7a7b-8d95-41f4-8528-2a2beb1a5021-c000.json
-rw-r--r--   2 root su

In [218]:
data = sc.textFile('hdfs:///user/transit/*/*/*.json')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [219]:
result=data.map(lambda t: json.loads(t)).flatMap(lambda x: x['entity']).map(
    lambda e: {'id':e['id'],'timestamp':e['vehicle']['timestamp']
               ,'latitude':e['vehicle']['position']['latitude'],'longitude':e['vehicle']['position']['longitude']
              , })

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [220]:
new_df=result.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [221]:
new_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timestamp: long (nullable = true)

In [222]:
new_df.createOrReplaceTempView('vehicles')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [226]:
spark.sql("select count(*) as num,min(timestamp) as mintime, max(timestamp) as maxtime from vehicles").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------+
|   num|   mintime|   maxtime|
+------+----------+----------+
|890529|1587673743|1587747459|
+------+----------+----------+

In [127]:
fdf=result.select('entity.id','entity.vehicle.timestamp',
               'entity.vehicle.position.longitude','entity.vehicle.position.latitude')
#,'entity.vehicle.position.latitude','entity.vehicle.position.longitude')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'PipelinedRDD' object has no attribute 'select'
Traceback (most recent call last):
AttributeError: 'PipelinedRDD' object has no attribute 'select'



In [209]:
sv=spark.sql("select timestamp,latitude,longitude from  vehicles ")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"cannot resolve '`latitude`' given input columns: [vehicles.id, vehicles.lat, vehicles.long, vehicles.timestamp]; line 1 pos 17;\n'Project [timestamp#141L, 'latitude, 'longitude]\n+- SubqueryAlias `vehicles`\n   +- LogicalRDD [id#138, lat#139, long#140, timestamp#141L], false\n"
Traceback (most recent call last):
  File "/tmp/nm-local-dir/usercache/root/appcache/application_1586890731024_0066/container_1586890731024_0066_01_000001/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/tmp/nm-local-dir/usercache/root/appcache/application_1586890731024_0066/container_1586890731024_0066_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/tmp/nm-local-dir/usercache/root/appcache/application_1586890731024_0066/container_1586890731024_0066_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in d

In [75]:
sv.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                           timestamp  ...                                          longitude
0  [1587667922, 1587667909, 1587667917, 158766791...  ...  [-76.6112, -76.61613, -76.60929, -76.555046, -...

[1 rows x 3 columns]